<a href="https://colab.research.google.com/github/firdaaacy/Praktikum-PPM/blob/main/Praktikum%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 3. Klasifikasi K-Nearest Neighbor (KNN)


Tuliskan Nama, NIM, dan kelas Anda:

Nama  : Firda Adha Cahyanti

NIM   : 195150201111037

Kelas : PPM - TIF F

### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [1]:
#mengunduh dataset iris pada alamat yang sudah diberikan pada modul
! wget https://dataset-ppm.s3.amazonaws.com/iris.csv

--2021-07-08 12:42:29--  https://dataset-ppm.s3.amazonaws.com/iris.csv
Resolving dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)... 54.231.114.219
Connecting to dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)|54.231.114.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4609 (4.5K) [text/csv]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   4.50K  --.-KB/s    in 0s      

2021-07-08 12:42:29 (118 MB/s) - ‘iris.csv’ saved [4609/4609]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [2]:
#melakukan import library yang akan digunakan
import pandas as pd #pemanggilan library pandas dengan alias pemanggilan pd
import numpy as np  #pemanggilan library numpy dengan alias pemanggilan np

#pembacaan dataset iris yang sudah di download menggunakan pandas
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [3]:
#menampilkan 5 data teratas dataset menggunakan .head()
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [4]:
#melakukan import sebagian library yang akan digunakan
#pada kasus  ini hanya mengimport train_test_split dari modul sklearn.model_selection
from sklearn.model_selection import train_test_split
#melakukan pembagian dataset dengan pembagian 20% data uji dan 80% data latih
data_latih, data_uji = train_test_split(data, test_size = 0.2, train_size = 0.8)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [5]:
#menampilkan jumlah baris data uji yang sudah dibagi (20%) dari dataset
print(data_uji.shape[0])
#menampilkan jumlah baris data latih yang sudah dibagi (80%) dari dataset
print(data_latih.shape[0])

30
120


Pisahkan label/kategori dari data latih dan data uji menjadi variabel tersendiri. Beri nama variabelnya **label_latih** dan **label_uji**

In [6]:
#melakukan pemisahan antara fitur dan kelas dengan melakukan pop pada kolom spesies pada label latih
label_latih = data_latih.pop('species')
#melakukan pemisahan antara fitur dan kelas dengan melakukan pop pada kolom spesies pada label uji
label_uji = data_uji.pop('species')

In [7]:
#menampilkan 5 baris teratas data latih yang sudah dipisahkan dengan labelnya
data_latih.head()

,sepal_length,sepal_width,petal_length,petal_width
127,6.1,3.0,4.9,1.8
123,6.3,2.7,4.9,1.8
60,5.0,2.0,3.5,1.0
144,6.7,3.3,5.7,2.5
76,6.8,2.8,4.8,1.4


In [8]:
#menampilkan 5 baris teratas data uji yang sudah dipisahkan dengan labelnya
data_uji.head()

,sepal_length,sepal_width,petal_length,petal_width
65,6.7,3.1,4.4,1.4
111,6.4,2.7,5.3,1.9
77,6.7,3.0,5.0,1.7
9,4.9,3.1,1.5,0.1
23,5.1,3.3,1.7,0.5


## 3) Menghitung jarak euclidean

Tahapan awal dari algoritma KNN adalah perhitungan jarak. Salah satu metode perhitungan jarak yang bisa digunakan adalah jarak Euclidean. Buatlah fungsi bernama **jarakEu** yang berfungsi menghitung jarak euclidean dari dua buah vektor (tanpa kelas)

In [9]:
#membuat fungsi jarakEu untuk mengitung jarak menggunakan algoritma Euclidean Distance
def jarakEu(data1,data2):
  #menghitung selisih kuadrat antar 2 data dengan fungsi square pada library numpy
  jarak = np.square(data1-data2)
  #menghitung jumlah jarak kuadrat antar 2 data dengan fungsi sum pada library numpy
  jarak = np.sum(jarak)
  #mengembalikan nilai akar dari jarak yang sudah dihitung
  return np.sqrt(jarak)

Uji fungsi **jarakEu** untuk menghitung jarak antara data latih pertama dengan data uji pertama.

In [10]:
#menghitung jarak data latih dan data uji pada indeks lokasi ke 0 (baris pertama) menggunakan fungsi jarakEU
jarak = jarakEu(data_latih.iloc[0], data_uji.iloc[0])
#menampilkan jarak yang didapatkan
print(jarak)

0.8831760866327851


## 4) Algoritma KNN

Implementasikan algoritma KNN dengan tahapan-tahapan berikut :
1. Hitung jarak antara data uji dengan setiap data latih.
2. Cari *k* data latih dengan jarak terkecil.
3. Tentukan kelas dari *k* data latih tersebut.
4. Kelas data uji ditentukan dari mayoritas kelas *k* data latih.

Buatlah fungsi dengan nama **knn** yang menerima input berupa *k*, sekumpulan data latih dan labelnya, serta sebuah data uji

In [11]:
#melakukan import sebagian library yang akan digunakan
#pada kasus  ini hanya mengimport Counter dari modul collections
from collections import Counter

#membuat fungsi KNN untuk memprediksikan data latih yang membutuhkan 4 parameter (nilai K, datalatih
#, labellatih dan datauji)
def knn(k ,datalatih, labellatih, datauji):
  #mengitung jarak dengan memanggil fungsi jarakEu dan disimpan dalam array
  jarak = np.array([jarakEu(datalatih.iloc[x],datauji) for x in range (datalatih.shape[0])])
  #mengambil indeks dalam array yang nilai jaraknya k teratas
  indeks_k_minimum = jarak.argsort()[:k]
  #mengambil data kelas/label dari data latih yang meliki jarak terdekat 
  k_kelas = labellatih.iloc[indeks_k_minimum].to_numpy()
  #melakukan penghitungan jumlah kelas dalam K
  counter = Counter(k_kelas)
  #menentukan kelas uji dimana counter kelas yang berada pada K paling  besar dan minimal ada 1 kali.
  kelas_uji = counter.most_common(1)[0][0]
  return kelas_uji

Lakukan pengujian fungsi **knn** untuk menentukan kelas dari data uji pertama

In [12]:
hasil = knn(3,data_latih,label_latih,data_uji.iloc[0])
print(hasil)

Iris-versicolor


## 5) KNN pada Scikit-learn

Library scikit-learn memiliki algoritma KNN yang siap untuk digunakan. Sebagai pengenalan, Anda akan melakukan klasifikasi data yang sama menggunakan KNN yang tersedia pada library scikit-learn

In [13]:
#melakukan import sebagian library yang akan digunakan
#pada kasus  ini hanya mengimport KNeighborsClassifier dari modul klearn.neighbors
from sklearn.neighbors import KNeighborsClassifier
#melakukan modelling KNN menggunakan jumlah tetangga k = 3
KNN = KNeighborsClassifier(n_neighbors = 3)
#menentukan data train beserta label menggunakan library KNN
KNN.fit(data_latih,label_latih)
#melakukan prediksi pada data uji pada indeks 0
kelas = KNN.predict([data_uji.iloc[0]])
#menampilkan kelas data yang diuji
print(kelas)

['Iris-versicolor']


## TUGAS
1. Buatlah fungsi bernama **jarakCity** yang berfungsi menghitung jarak City Block antara dua vektor.
2. Buatlah sebuah fungsi bernama **knn_multi** yang dapat menentukan kelas dari **semua** data uji (Fungsi **knn** yang sudah Anda buat hanya dapat menentukan kelas dari sebuah data uji. Anda dapat memanfaatkan fungsi **knn** yang telah ada
3. Buatlah fungsi bernama **cek_hasil** yang berfungsi membandingkan hasil klasifikasi dengan label sebenarnya. Fungsi ini menerima input 2 buah list, yaitu list label hasil klasifikasi dan list label data uji yang sebenarnya. Output dari fungsi ini adalah integer, yang menunjukkan berapa banyak label hasil klasifikasi yang sama dengan label data uji sebenarnya.
4. Jalankan fungsi **knn_multi** dengan nilai k=3, k=7,k=9 dan k=13. Amati hasil dari fungsi **cek_hasil** dengan nilai k tersebut. Apa kesimpulan Anda?

In [14]:
#membuat fungsi jarakCity untuk mengitung jarak menggunakan algoritma Manhattan Distance
def jarakCity(data1,data2):
  jarak = np.abs(data1-data2);
  jarak = np.sum(jarak);
  return jarak

In [15]:
jarakManhattan = jarakCity(data_latih.iloc[0], data_uji.iloc[0])
jarakManhattan

1.6000000000000008

In [16]:
#melakukan import sebagian library yang akan digunakan
#pada kasus  ini hanya mengimport Counter dari modul collections
from collections import Counter

#membuat fungsi KNN untuk memprediksikan data latih yang membutuhkan 4 parameter (nilai K, datalatih
#, labellatih dan datauji)
def knnCity(k ,datalatih, labellatih, datauji):
  #mengitung jarak dengan memanggil fungsi jarakCity dan disimpan dalam array
  jarak = np.array([jarakCity(datalatih.iloc[x],datauji) for x in range (datalatih.shape[0])])
  #mengambil indeks dalam array yang nilai jaraknya k teratas
  indeks_k_minimum = jarak.argsort()[:k]
  #mengambil data kelas/label dari data latih yang meliki jarak terdekat 
  k_kelas = labellatih.iloc[indeks_k_minimum].to_numpy()
  #melakukan penghitungan jumlah kelas dalam K
  counter = Counter(k_kelas)
  #menentukan kelas uji dimana counter kelas yang berada pada K paling  besar dan minimal ada 1 kali.
  kelas_uji = counter.most_common(1)[0][0]
  return kelas_uji

In [17]:
hasil = knnCity(3,data_latih,label_latih,data_uji.iloc[0])
print(hasil)

Iris-versicolor


In [18]:
#jawaban soal 2
#membuat fungsi KNN MULTI untuk menenetukan semua kelas data uji menggunakan Euclidean Distance
def knn_multi_Eu(k, datalatih, labellatih, datauji):
  #membuat variabel list result yang digunakan untuk menampung hasil kelas tiap data uji
  result = []
  #melakukan perulangan setiap data latih 
  for data in range(datauji.shape[0]):
    #melakukan pengujian setiap data uji terhadap data latih
    kelas = knn(k,datalatih,labellatih,datauji.iloc[data])
    #menambahkan hasil kelas setiap data uji 
    result.append(kelas)
  return result

In [19]:
#membuat fungsi KNN MULTI untuk menenetukan semua kelas data uji menggunakan Manhattan Distance
def knn_multi_Man(k, datalatih, labellatih, datauji):
  #membuat variabel list result yang digunakan untuk menampung hasil kelas tiap data uji
  result = []
  #melakukan perulangan setiap data latih 
  for data in range(datauji.shape[0]):
    #melakukan pengujian setiap data uji terhadap data latih
    kelas = knnCity(k,datalatih,labellatih,datauji.iloc[data])
    #menambahkan hasil kelas setiap data uji 
    result.append(kelas)
  return result

In [20]:
#melakukan pengujian untuk semua data uji menggunakan KNN euclidean distance
listEu = knn_multi_Eu(3,data_latih,label_latih,data_uji)

In [21]:
#melakukan pengujian untuk semua data uji menggunakan KNN Manhattan distance
listMan = knn_multi_Man(3,data_latih,label_latih,data_uji)

In [22]:
#jawaban soal 3
#membuat fungsi cek-hasil untuk membandingkan antara label hasil knn dan label sebenarnya
def cek_hasil(k,label_hasil_pengujian, label_uji):
  #variabel untuk menghitung berapa data yang sudah sesuai
  HasilSama = 0;
  for i in range(len(label_hasil_pengujian)):
    #kondisi jika hasil pengujian dan hasil sebenarnya sama
    if (label_hasil_pengujian[i] == label_uji.iloc[i]):
      #penambahan nilai 1 jika hasilnya sama
      HasilSama +=1
  
  #cetak jumlah K yang digunakan
  print("Jumlah K dalam pengujian : ", k)
  #cetak jumlah label yang diuji
  print("Jumlah Label yang diuji : ", len(label_hasil_pengujian))
  #cetak jumlah label yang sudah benar
  print("Jumlah Label yang sama dengan data asli : ", HasilSama)
  #cetak akurasi
  print("Akurasi = ", HasilSama/len(label_hasil_pengujian))

In [23]:
#Pengecekan akurasi dengan k= 3 metode euclidean distance
cek_hasil(3,listEu,label_uji)

Jumlah K dalam pengujian :  3
Jumlah Label yang diuji :  30
Jumlah Label yang sama dengan data asli :  30
Akurasi =  1.0


In [24]:
#Pengecekan akurasi dengan k= 3 metode Manhattan distance
cek_hasil(3,listMan,label_uji)

Jumlah K dalam pengujian :  3
Jumlah Label yang diuji :  30
Jumlah Label yang sama dengan data asli :  30
Akurasi =  1.0


In [25]:
#jawaban soal 4
#testing data dengan k= 3 metode euclidean distance
listEu = knn_multi_Eu(3,data_latih,label_latih,data_uji)
#testing data dengan k= 7 metode euclidean distance
listEu7 = knn_multi_Eu(7,data_latih,label_latih,data_uji)
#testing data dengan k= 9 metode euclidean distance
listEu9 = knn_multi_Eu(9,data_latih,label_latih,data_uji)
#testing data dengan k= 13 metode euclidean distance
listEu13 = knn_multi_Eu(13,data_latih,label_latih,data_uji)

In [26]:
#Pengecekan akurasi dengan k= 3 metode euclidean distance
cek_hasil(3,listEu,label_uji)

Jumlah K dalam pengujian :  3
Jumlah Label yang diuji :  30
Jumlah Label yang sama dengan data asli :  30
Akurasi =  1.0


In [27]:
#Pengecekan akurasi dengan k= 7 metode euclidean distance
cek_hasil(7,listEu7,label_uji)

Jumlah K dalam pengujian :  7
Jumlah Label yang diuji :  30
Jumlah Label yang sama dengan data asli :  29
Akurasi =  0.9666666666666667


In [28]:
#Pengecekan akurasi dengan k= 9 metode euclidean distance
cek_hasil(9,listEu9,label_uji)

Jumlah K dalam pengujian :  9
Jumlah Label yang diuji :  30
Jumlah Label yang sama dengan data asli :  29
Akurasi =  0.9666666666666667


In [29]:
#Pengecekan akurasi dengan k= 13 metode euclidean distance
cek_hasil(13,listEu13,label_uji)

Jumlah K dalam pengujian :  13
Jumlah Label yang diuji :  30
Jumlah Label yang sama dengan data asli :  29
Akurasi =  0.9666666666666667


Penjelasan jawaban soal nomor 4

KNN atau K-Nearest Neighbor merupakan salah satu metode klasifikasi data dengan membandingkan jarak terdekat. Nilai $K$ yang digunakan adalah jumlah tetangga terdekat yang akan dibandingkan. Setelah melakukan perbandingan dengan tetangga terdekat maka akan dilakukan voting label manakah yang memiliki jumlah terbanyak dan label itulah yang dimiliki data test. Sehingga pemilihan jumlah nilai K berpengaruh pada akurasi metode ini.

Dapat dilihat pada data pengecekan ketika $K$ bernilai 3 maka akurasinya sebesar 0.9, sedangkan pada $K$ = 7 sebesar 0.933, pada $K$ = 9 sebesar 1 (semua data tes benar), dan 0,966 pada $K$ = 13. Pada metode ini harus menghindari jumlah $K$ yang bernilai genap dikarenakan nilai voting dapat seri antar label sehingga dapat menurunkan akurasi. Sedangkan jika menggunakan jumlah $k$ ganjil, hal tersebut dapat dihindari atau memiliki potensi lebih rendah mendapati kasus tersebut. Meskipun demikian, pembagian data latih dan tes dapat mempengaruhi keakuratan, kualitas data, dan metode penghitungan jarak (seperti manhattan, euclidean) dan lain-lain.